# 🧪 Experiment: Gazetteer Length Group
This notebook evaluate the test set for the task `Gazetteer Length Group`.

**Note**: Before conducting experiments, you need to install `kaner` package first. Otherwise, this notebook will raise an *import error*.

```bash
cd ../
python setup.py install
```

In [ ]:
import json
import os
import gc
from copy import deepcopy
from collections import defaultdict
from typing import List, Dict, Any
from datetime import datetime
import pprint

import tqdm
import torch.nn as nn

from kaner.context import GlobalContext as gctx
from kaner.adapter.tokenizer import CharTokenizer
from kaner.adapter.knowledge import Gazetteer
from kaner.adapter.in_adapter import split_dataset
from kaner.adapter.out_adapter import BaseOutAdapter
from kaner.trainer import NERTrainer, TrainerConfig
from kaner.tracker import NERTracker, NERTrackerRow
from kaner.common import load_json, load_jsonl, save_json
from kaner.common.func import query_time


gctx.init()

## 1. Training Function

In [ ]:
def get_masked_gazetteer_by_length(len_start: int, len_end: int, gazetteer_folder: str) -> Gazetteer:
    gazetteer = Gazetteer(gazetteer_folder)
    lexicons = []
    lexicon_sets = defaultdict(list)
    with open(os.path.join(gazetteer_folder, "lexicons.txt"), "r", encoding="utf-8") as fin:
        for line in fin.readlines():
            lexicon = line.replace("\n", "").split("\t")[0]
            if lexicon == "[PAD]":
                continue
            lexicons.append(lexicon)
            lexicon_sets[len(lexicon)].append(lexicon)
    remaining_lexicons = set()
    for i in range(len_start, len_end + 1):
        remaining_lexicons.update(lexicon_sets[i])
    # keep the same number of lexicons
    remaining_lexicons = set(list(remaining_lexicons)[:200])
    masked_lexicons = list(set(lexicons) - remaining_lexicons)
    print("Total: {0}, Remaining: {1}, Masked: {2}".format(len(lexicons), len(remaining_lexicons), len(masked_lexicons)))
    gazetteer.mask(masked_lexicons, True)

    return gazetteer


def train(config: TrainerConfig, len_start: int, len_end: int) -> Dict[str, Any]:
    """
    Given a configuration, train a model on a dataset with gazetteer modification.

    Args:
        config (TrainerConfig): Trainer Configuration.
    """

    def update_hyperparameters(tokenizer: CharTokenizer, out_adapter: BaseOutAdapter, gazetteer: Gazetteer):
        """
        Update hyper parameters.

        Args:
            tokenizer (CharTokenizer): Tokenizer.
            out_adapter (BaseOutAdapter): Output adapter.
            gazetteer (Gazetteer): Gazetteer.
        """
        partial_configs = {"n_tags": len(out_adapter)}
        partial_configs.update(tokenizer.configs())
        partial_configs.update(gazetteer.configs())

        return partial_configs

    raw_datasets = split_dataset(config.dataset_folder, dataset_pp=config.dataset_pp)
    tokenizer = CharTokenizer(config.tokenizer_model_folder)
    tokenizer.save(config.output_folder)
    gazetteer = get_masked_gazetteer_by_length(len_start, len_end, config.gazetteer_model_folder)
    gazetteer.save(config.output_folder)
    out_adapter = gctx.create_outadapter(config.out_adapter, dataset_folder=config.dataset_folder, file_name="labels")
    out_adapter.save(config.output_folder, "labels")
    in_adapters = (
        gctx.create_inadapter(
            config.in_adapter, dataset=dataset, tokenizer=tokenizer, out_adapter=out_adapter, gazetteer=gazetteer,
            **config.hyperparameters
        )
        for dataset in raw_datasets
    )
    token_embeddings = tokenizer.embeddings()
    lexicon_embeddings = gazetteer.embeddings()
    config.hyperparameters = update_hyperparameters(tokenizer, out_adapter, gazetteer)
    collate_fn = gctx.create_batcher(
        config.model, input_pad=tokenizer.pad_id, output_pad=out_adapter.unk_id, lexicon_pad=gazetteer.pad_id, device=config.device
    )
    model = gctx.create_model(config.model, **config.hyperparameters, token_embeddings=token_embeddings, lexicon_embeddings=lexicon_embeddings)
    trainer = NERTrainer(
        config, tokenizer, in_adapters, out_adapter, collate_fn, model, nn.CrossEntropyLoss(),
        gctx.create_traincallback(config.model), gctx.create_testcallback(config.model)
    )
    results = trainer.train()

    return results, trainer

In [ ]:
def trainall(labpath: str, cfgdir: str, m: List[str], d: List[str], n: int, **kwargs) -> None:
    """
    Experiments for all model's training.

    Args:
        labpath (str): The file path of recording experimental results.
        cfgdir (str): Configuration folder.
        m (List[str]): All specific models to be trained.
        d (List[str]): All specific datasets to be tested.
        n (int): The number of training repeating times.
        tag (str): Experimental tags.
    """

    def update_names(names: List[str], all_names: List[str], name_type: str) -> List[str]:
        """
        Check whether the name that user inputs is correct.

        Args:
            names (List[str]): The names (dataset, model, gazetteer) that user inputs.
            all_names (List[str]): All names (dataset, model, gazetteer) that this libary provides.
            name_type (str): The type of the name (Dataset, Model, Gazetteer).
        """
        if len(names) == 0:
            names = all_names
        else:
            for name in names:
                if name not in all_names:
                    print("[{0}] {1} is not in {2}".format(name_type, name, all_names))
                    exit(0)
        return names

    tracker = NERTracker.load(labpath)
    models = update_names(m, gctx.get_model_names(), "Model")
    datasets = update_names(d, gctx.get_dataset_names(), "Dataset")

    print("--------------------- Laboratory Configuration ---------------------")
    print("Models: {0}".format(models))
    print("Datasets: {0}".format(datasets))
    print("--------------------------------------------------------------------")

    for dataset in datasets:
        for model in models:
            for gazetteer_model in ["gigaword", "sgns", "tec"]:
                for _ in range(n):
                    for len_start, len_end in [(2, 3), (4, 5), (6,7)]:
                        tag = "{0}-{1}".format(len_start, len_end)
                        if len(tracker.query(dataset=dataset, model=model, gazetteer_model=gazetteer_model, tag=tag)) >= n:
                            continue
                        config = TrainerConfig(os.path.join(cfgdir, model + ".yml"), dataset=dataset, gazetteer_model=gazetteer_model, **kwargs)
                        start = str(datetime.now())
                        try:
                            results, trainer = train(config, len_start, len_end)
                        except RuntimeError as error:
                            print(error)
                            continue
                        tracker.insert(
                            NERTrackerRow(
                                start, model, dataset, config.tokenizer_model, gazetteer_model, config.output_folder, query_time(trainer.train),
                                results["f1-score"], results["precision-score"], results["recall-score"], results["epoch_count"], results["test-loss"], tag
                            )
                        )
                        tracker.save(labpath)
                        del trainer

## 2. Experiments

In [ ]:
labpath = "../data/logs/experiments_gazetteer_length_group.csv"
cfgdir = "../configs"
models = ["ses", "cgn", "mdgg"]
datasets = ["weiboner"]
n = 4
kwargs = {"data_folder": "../data", "gpu": [0]}

trainall(labpath, cfgdir, models, datasets, n, **kwargs)